In [1]:
import numpy as np
from ondelette import *
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_probability as tfp

dtype = tf.float32
rng = np.random.default_rng()

2025-03-20 17:54:19.294205: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-20 17:54:19.297950: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-20 17:54:19.311004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-20 17:54:19.336550: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-20 17:54:19.336598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-20 17:54:19.355070: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
song = 'MusicDelta_Rock'
audio = tfio.audio.AudioIOTensor(f'data/MedleyDB_V1/{song}_MIX.mp3', dtype=tf.float32)
rate = tf.cast(audio.rate, dtype=tf.int64)
raw_samples = audio.to_tensor()
samples = tf.transpose(raw_samples)
print(f'samples: {samples.shape[1]}')


2025-03-20 17:54:24.299081: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA


samples: 577320


In [18]:
def interp(new_length, frames):
    dtype=frames[0].dtype
    old_length = frames[0].shape[0]
    x = tf.linspace(tf.constant(0, dtype=dtype), tf.constant(old_length, dtype=dtype), new_length)
    print(f'{list(x.numpy())}')

    new_frames = []
    for level in frames:
        new = tfp.math.interp_regular_1d_grid(
            x=x,
            x_ref_min = 0,
            x_ref_max = old_length,
            y_ref = level,
            axis = 0)
        new_frames.append(new)
    return new_frames

for name, make in wavelets.all.items():
    wavelet = make(dtype)
    filters = wavelets.dwt_filters(wavelet)
    inverse_filters = wavelets.idwt_filters(wavelet)

    dec = wavelets.wavedec_valid(filters, 10, samples)
    print(f'Levels: {len(dec)}')

    frames = framing.levels_to_frames(dec)
    old_length = frames[0].shape[0]
    new_length = (old_length * 2) + 1
    slow = framing.stretch_linear(new_length, frames)
    deframed = framing.frames_to_levels(slow)
    rec = wavelets.waverec(inverse_filters, deframed)

    tx = tf.transpose(rec)
    mp3 = tfio.audio.encode_mp3(tx, rate)
    tf.io.write_file(f'temp/slow/{name}.mp3', mp3)


Levels: 10
[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0, 28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5, 35.0, 35.5, 36.0, 36.5, 37.0, 37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0, 52.5, 53.0, 53.5, 54.0, 54.5, 55.0, 55.5, 56.0, 56.5, 57.0, 57.5, 58.0, 58.5, 59.0, 59.5, 60.0, 60.5, 61.0, 61.5, 62.0, 62.5, 63.0, 63.5, 64.0, 64.5, 65.0, 65.5, 66.0, 66.5, 67.0, 67.5, 68.0, 68.5, 69.0, 69.5, 70.0, 70.5, 71.0, 71.5, 72.0, 72.5, 73.0, 73.5, 74.0, 74.5, 75.0, 75.5, 76.0, 76.5, 77.0, 77.5, 78.0, 78.5, 79.0, 79.5, 80.0, 80.5, 81.0, 81.5, 82.0, 82.5, 83.0, 83.5, 